In [20]:
from datetime import datetime
from google.cloud import bigquery
import os
import sys
import time
import pandas as pd
import pandas_gbq

In [38]:
def get_csv_files(path):
        files = os.listdir(path)
        files = [f for f in files if f.startswith("raw") and f.endswith(".csv")]
        files = map(lambda x: os.path.join(path, x), files)
        return sorted(files)

    
def get_date(folder="./output"):
    content = [fn for fn in os.listdir(folder)]
    dates = [datetime.strptime(fn, "%Y%m%d_%H%M%S") for fn in content]
    return dates[dates.index(max(dates))].strftime("%Y%m%d_%H%M%S")

class bqLoader():
    def __init__(self, credentials=None, path=None, table_id=None, dataset=None):
        self.credentials = credentials or [".gcpkey/"+fn for fn in os.listdir(".gcpkey") if fn.endswith(".json")][0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = self.credentials
        self.client    = bigquery.Client()
        self.path      = path or "output/{}/rawedges".format(get_date())
        self.table_id  = table_id or "btc"
        self.dataset   = dataset or "bitcoin_transactions"
    
    
    def upload_data(self):
        files = get_csv_files(self.path)
        r = len(files)
        p = lambda x: "\r[" + "#" * x + (r - x) * " " + "]"
        for i, blkfile in enumerate(files):
            df = pd.read_csv(blkfile, names=["ts", "from", "to"])
            #df.to_gbq(self.table_id+"."+self.dataset, if_exists="append", chunksize=int(1e7))
            time.sleep(2)
            sys.stdout.write("\r\r{:<18} successfully uploaded   \n".format(blkfile.split("/")[-1]))            
            sys.stdout.write(p(i+1))
            time.sleep(0.1)


In [5]:
1e6 == 1000000

True

In [16]:
get_date()

'20211007_092616'

In [ ]:
pd.read_csv(blkfile, names=["ts", "from", "to"])

In [39]:
a = bqLoader()
a.upload_data()

raw_blk_0.csv      successfully uploaded   
raw_blk_0_.csv     successfully uploaded   
raw_blk_1.csv      successfully uploaded   
[###]

In [5]:
a = pd.read_csv("output/{}/rawedges".format(get_date())+"/raw_blk_0.csv", names=["ts", "from", "to"])[:10000]

In [6]:
a.to_csv("output/{}/rawedges".format(get_date())+"/raw_blk_0_.csv", index=None, header=None)

In [2]:
get_files("data")

['data/blk00000.dat',
 'data/blk00001.dat',
 'data/blk00002.dat',
 'data/blk00003.dat']